In [1]:
system.file(package = "simDNAmixtures")

[1] "/Library/Frameworks/R.framework/Versions/4.5-x86_64/Resources/library/simDNAmixtures"

In [ ]:
# Load packages
library(simDNAmixtures)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
set.seed(1)

# Load example allele frequency file that comes with the package
allele_freqs_file <- system.file("extdata","FBI_extended_Cauc_022024.csv",
                                       package = "simDNAmixtures")

# Read allele frequencies to create an empirical distribution
allele_freqs <- read_allele_freqs(allele_freqs_file)

# Configuration for the GLobalFiler kit
gf <- gf_configuration()

In [4]:
genotypes <- sample_contributor_genotypes(
    contributors = c("U1", "U2"),
    loci = gf$log_normal_settings$locus_names,
    freqs = allele_freqs
    )
# genotypes

In [6]:
parameter_summary <- data.frame(
  SampleName =  "someonesaidsomething",
  contributors1 = "U1",
  contributors2 = "U2",
  model = "log_normal_model",
  template1 = 10000,
  template2 = 20000,
  degradation1 = 0.001,
  degradation2 = 0.002,
  c2 = 14.754,
  k2BackStutter = 14.48,
  k2ForwardStutter = 9.67,
  k22bpBackStutter = 3.13,
  k2DoubleBackStutter = 6.97
)
# Add LSAE columns
LSAE <- sample_LSAE(LSAE_variance = gf$log_normal_settings$LSAE_variance_prior,
                             locus_names = gf$log_normal_settings$locus_names)
for (locus in gf$log_normal_settings$locus_names) {
  parameter_summary[[locus]] <- LSAE[locus]
}
parameter_summary

SampleName,contributors1,contributors2,model,template1,template2,degradation1,degradation2,c2,k2BackStutter,⋯,FGA,D22S1045,D5S818,D13S317,D7S820,SE33,D10S1248,D1S1656,D12S391,D2S1338
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
someonesaidsomething,U1,U2,log_normal_model,10000,20000,0.001,0.002,14.754,14.48,⋯,1.874905,0.8899836,0.7179199,1.198203,0.9580408,2.143089,0.9876229,1.244727,1.008927,0.7898541


In [7]:
mixtures <- sample_mixtures_fixed_parameters(
  genotypes = genotypes,
  parameter_summary = parameter_summary,
  model_settings = gf$log_normal_settings,
)

In [8]:
# names(mixtures$samples[[1]]$annotated_mixture)
# mixtures$samples[[1]]$annotated_mixture

In [9]:
# mixtures$samples[[1]]$contributor_genotypes

## Known Genotype

In [14]:
parameter_summary_file <- utils::read.csv(system.file(
    "extdata","Example_2p_Parameter_Summary.csv",
    package = "simDNAmixtures"))

k22bpBackStutter <- 3.13
k2DoubleBackStutter <- 6.97
# Add the above two k2 values to the parameter summary in two new columns
parameter_summary_file$k22bpBackStutter <- k22bpBackStutter
parameter_summary_file$k2DoubleBackStutter <- k2DoubleBackStutter

knitr::kable(parameter_summary_file)



|SampleName                  |contributors1 |contributors2 |model            | template1| template2| degradation1| degradation2|       c2| k2BackStutter| k2ForwardStutter|   D3S1358|       vWA|   D16S539|    CSF1PO|      TPOX|      AMEL|   D8S1179|    D21S11|    D18S51|    D2S441|   D19S433|      TH01|       FGA|  D22S1045|    D5S818|   D13S317|    D7S820|      SE33|  D10S1248|   D1S1656|   D12S391|   D2S1338| k22bpBackStutter| k2DoubleBackStutter|
|:---------------------------|:-------------|:-------------|:----------------|---------:|---------:|------------:|------------:|--------:|-------------:|----------------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------------:|-------------------:|
|sample_0001_N2_358_1355_ab  |K1            |K2            |log_normal_model |  357.6732|  1

In [10]:
# genotypes_file_summary
genotype_file <- system.file("extdata","Example_References_DB.csv",
                                       package = "simDNAmixtures")
db_refs <- utils::read.csv(genotype_file, check.names = FALSE)
known_genotypes <- simDNAmixtures:::.wide_references_to_allele_tables(db_refs)

In [11]:
file_mixtures <- sample_mixtures_fixed_parameters(
  genotypes = known_genotypes,
  parameter_summary = parameter_summary_file,
  model_settings = gf$log_normal_settings,
)

In [ ]:
knitr::kable(file_mixtures$samples[[1]])



|SampleName                 |Marker   |Allele |   Size| Height|   Expected|      LSAE| ExpectedAllele| ExpectedAllelePreStutter| SumOfStutterRatios| ExpectedStutter| StutterRatioBackStutter|StutterParentBackStutter |StutterProductBackStutter | StutterRatioForwardStutter|StutterParentForwardStutter |StutterProductForwardStutter | StutterRatio2bpBackStutter|StutterParent2bpBackStutter |StutterProduct2bpBackStutter | StutterRatioDoubleBackStutter|StutterParentDoubleBackStutter |StutterProductDoubleBackStutter | ExpectedBackStutter| ExpectedForwardStutter| Expected2bpBackStutter| ExpectedDoubleBackStutter|       c2| VarianceAllele| HeightAllele| StutterCapBackStutter| StutterCapForwardStutter| StutterCap2bpBackStutter| StutterCapDoubleBackStutter| HeightUncappedBackStutter| VarianceBackStutter| HeightUncappedForwardStutter| VarianceForwardStutter| HeightUncapped2bpBackStutter| Variance2bpBackStutter| HeightUncappedDoubleBackStutter| VarianceDoubleBackStutter| HeightUncappedStutter| Stutt

# FULL FILE

In [13]:
library(simDNAmixtures)
library(dplyr)

# Output directory
out_dir <- "generated_alleles_fixed_params"
dir.create(out_dir, showWarnings = FALSE)

# Load allele frequencies
allele_freqs_file <- system.file("extdata","FBI_extended_Cauc_022024.csv", package = "simDNAmixtures")
allele_freqs <- read_allele_freqs(allele_freqs_file)
gf <- gf_configuration()

# # Filter loci to only those supported by the size regression
# supported_loci <- names(gf$size_regression)
# gf$log_normal_settings$locus_names <- intersect(gf$log_normal_settings$locus_names, supported_loci)

dye_map <- kits$GlobalFiler[, c("Marker", "Color")] %>% distinct(Marker, .keep_all = TRUE)
size_standard_sizes <- c(20, 40, 60, 80, 100, 114, 120, 140, 160, 180, 200, 214,
                         220, 240, 250, 260, 280, 300, 314, 320, 340, 360, 380,
                         400, 414, 420, 440, 460, 480, 500, 514, 520, 540, 560,
                         580, 600)
size_standard_df <- data.frame(
  Locus = "LIZ",
  Allele = NA,
  Height = 3000,
  Size = size_standard_sizes,
  Color = "orange"
)

# Template ratio functions
template_ratio_even <- function(n) rep(1/n, n)
template_ratio_increasing <- function(n) { ratios <- 1:n; ratios / sum(ratios) }
template_ratio_last_dominates <- function(n) { ratios <- rep(1/20, n); ratios[n] <- 1 - sum(ratios[-n]); ratios }
template_ratio_first_two_10 <- function(n) {
  ratios <- rep(0, n)
  if (n == 1) {
    ratios[1] <- 1
  } else if (n == 2) {
    ratios[1] <- 0.1
    ratios[2] <- 0.9
  } else {
    ratios[1:2] <- 0.1
    ratios[3:n] <- (1 - 0.2) / (n - 2)
  }
  ratios
}
template_ratio_functions <- list(
  template_ratio_even,
  template_ratio_increasing,
  template_ratio_last_dominates,
  template_ratio_first_two_10
)

# Varying parameters
template_amounts <- c(100, 1000)  # total template amount per sample
degradation_amounts <- c(0.0025, 0.007)  # total degradation amount per sample

# Constants
c2 <- 14.754
k2BackStutter <- 14.48
k2ForwardStutter <- 9.67
k22bpBackStutter <- 3.13
k2DoubleBackStutter <- 6.97

# Randomly sampled parameters will be generated inside the loop



# Number of genotypes per configuration
n_samples <- 1
replicates <- 2
contributors_list <- c(2, 3, 4, 5)


# Must loop over the following elements:
#   - contributors_list
#   - template_amounts
#   - template_ratio_functions
#   - degradation_amounts
#   - n_samples
#   - replicates

# Main simulation loop - Contributors
for (n_contributors in contributors_list) {
    contributors <- paste0("U", seq_len(n_contributors))

  # Template amounts and ratios
  for (template_amount in template_amounts) {
    for (ratio_idx in seq_along(template_ratio_functions)) {
      ratio_func <- template_ratio_functions[[ratio_idx]]
      
      template_ratios <- ratio_func(n_contributors)
      templates <- template_amount * template_ratios

      # Degradation amounts
      for (degradation_idx in seq_along(degradation_amounts)) {
        degradation_value <- degradation_amounts[degradation_idx]
        # Per sample
        for (sample_idx in 1:n_samples) {

          # TODO: What if we generate genotypes in outermost loop? Will the models trained on it be worse?
          genotypes <- sample_contributor_genotypes(
            contributors = contributors,
            loci = gf$log_normal_settings$locus_names,
            freqs = allele_freqs
          )
          
          # Sample LSAE for this sample
          # TODO: Inside or outside replicate loop?
          LSAE <- sample_LSAE(LSAE_variance = gf$log_normal_settings$LSAE_variance_prior,
                             locus_names = gf$log_normal_settings$locus_names)

          # Per replicate
          for (rep in 1:replicates) {
            set.seed(1e7 * n_contributors + 1e5 * template_amount + 1e4 * ratio_idx + 1e3 * sample_idx + 1e2 * degradation_idx + rep)

            # # Create parameter summary for this sample (single row, all contributors as columns)
            # parameter_summary <- data.frame(
            #   SampleName = paste0(
            #     "nC", n_contributors, "_tmp", template_amount, "_ratio", ratio_idx,
            #     "_deg", degradation_idx, "_s", sample_idx, "_rep", rep
            #   ),
            #   # Contributors
            #   setNames(as.list(contributors), paste0("contributors", seq_len(n_contributors))),
            #   model = "log_normal_model",
            #   # Templates
            #   setNames(as.list(templates), paste0("template", seq_len(n_contributors))),
            #   # Degradation
            #   setNames(as.list(rep(degradation_value, n_contributors)), paste0("degradation", seq_len(n_contributors))),
            #   # c2, k2BackStutter, k2ForwardStutter, etc. (single values)
            #   c2 = c2,
            #   k2BackStutter = k2BackStutter,
            #   k2ForwardStutter = k2ForwardStutter,
            #   k22bpBackStutter = k22bpBackStutter,
            #   k2DoubleBackStutter = k2DoubleBackStutter
            # )

            # # Add LSAE columns
            # for (locus in gf$log_normal_settings$locus_names) {
            #   parameter_summary[[locus]] <- LSAE[locus]
            # }

            parameter_summary <- data.frame(
              SampleName =  "tegroe",
              setNames(as.list(contributors), paste0("contributors", seq_len(n_contributors))),
              model = "log_normal_model",
              template1 = 1000,
              template2 = 2000,
              degradation1 = 0.001,
              degradation2 = 0.002,
              c2 = 14.754,
              k2BackStutter = 14.48,
              k2ForwardStutter = 9.67,
              k22bpBackStutter = 3.13,
              k2DoubleBackStutter = 6.97
            )
            # Add LSAE columns
            for (locus in gf$log_normal_settings$locus_names) {
              parameter_summary[[locus]] <- LSAE[locus]
            }

            # Generated mixture
            mixture <- sample_mixtures_fixed_parameters(
              genotypes = genotypes,
              parameter_summary = parameter_summary,
              model_settings = gf$log_normal_settings
            )

            sim_peaks <- mixture$samples[[1]]$annotated_mixture
            sim_peaks_with_dye <- left_join(sim_peaks, dye_map, by = c("Locus" = "Marker"))
            combined_peaks_df <- rbind(sim_peaks_with_dye, size_standard_df)
            combined_peaks_df <- combined_peaks_df %>%
              mutate(Scan = round(Size * 11.2 + 3500))

            out_file <- sprintf(
              "%s/epg_nC%d_tmp%d_ratio%d_deg%d_s%d_rep%d.csv",
              out_dir, n_contributors, template_amount, ratio_idx, degradation_idx, sample_idx, rep
            )
            write.csv(combined_peaks_df, out_file, row.names = FALSE)
          }
        }
      }
    }
  }
}


ERROR: [1m[33mError[39m in `left_join()`:[22m
[1m[22m[33m![39m Join columns in `x` must be present in the data.
[31m✖[39m Problem with `Locus`.
